In [1]:
from selenium import webdriver

from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver.support.relative_locator import locate_with

import time
import openpyxl
import mysql.connector

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select


In [2]:
driver = webdriver.Chrome()
driver.get("http://localhost:4000/index1.php")
driver.minimize_window()
driver.maximize_window()


In [3]:
#pip install --user mysql-connector-python


In [4]:

# def connect_mysql(host, user, password, database):
#     try:
#         con = mysql.connector.connect(
#             host=host,
#             user=user,
#             password=password,
#             database=database
#         )
#         cursor = con.cursor()
#         cursor.execute("SELECT * FROM appointmenttb;")
#         tables = cursor.fetchall()
#         con.close()
#         return tables
#     except mysql.connector.Error as e:
#         print(f"Database error: {e}")
#         return None

# host = 'localhost'
# user = 'root'
# password = 'root'
# database = 'myhmsdb'
# tables = connect_mysql(host, user, password, database)
# if tables:
#     print("Tables in the database:", tables)
# else:
#     print("Failed to connect to the database or no tables found.")

In [5]:
# import mysql.connector

# def get_table_snapshot(conn, table_name):
#     cursor = conn.cursor()
#     cursor.execute(f"SELECT * FROM {table_name}")
#     return cursor.fetchall()

# def connect_mysql(host, user, password, database):
#     try:
#         conn = mysql.connector.connect(
#             host=host,
#             user=user,
#             password=password,
#             database=database
#         )
#         return conn
#     except mysql.connector.Error as e:
#         print(f"Database error: {e}")
#         return None

# # Configuration
# host = 'localhost'
# user = 'root'
# password = 'root'
# database = 'myhmsdb'
# table_name = 'appointmenttb'

# # Connect to your MySQL database
# conn = connect_mysql(host, user, password, database)
# if conn is None:
#     print("Failed to connect to the database.")
# else:
#     # Get snapshot before changes
#     before_snapshot = get_table_snapshot(conn, 'appointmenttb')
    
# #     # Perform some operations (INSERT, UPDATE, DELETE)
#     cursor = conn.cursor()
#     cursor.execute(f"TRUNCATE TABLE appointmenttb")
# #     conn.commit()  # Make sure to commit the transaction
    
#     # Get snapshot after changes
#     after_snapshot = get_table_snapshot(conn, 'appointmenttb')
    
#     # Compare snapshots
#     changes_made = before_snapshot != after_snapshot
    
#     if changes_made:
#         print("Changes were made.")
#     else:
#         print("No changes were made.")
    


In [6]:

def get_table_snapshot(conn, table_name):
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {table_name}")
    return cursor.fetchall()

def connect_mysql(host, user, password, database):
    try:
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        return conn
    except mysql.connector.Error as e:
        print(f"Database error: {e}")
        return None

# Configuration
host = 'localhost'
user = 'root'
password = 'root'
database = 'myhmsdb'
table_name = 'appointmenttb'

# Connect to your MySQL database
conn = connect_mysql(host, user, password, database)


excell_path = r'C:\Users\deeks\OneDrive\Documents\Desktop\record.xlsx'
workbook = openpyxl.load_workbook(excell_path)
sheet = workbook.active

#initialising successful and unsucessful cases
successfull = 0
unsuccessfull = 0

for row in sheet.iter_rows(min_row=2, values_only=True):

    global successfull ,unsuccessfull    

    email = row[0]
    password = row[1]
    specialisation = row[2]
    doctor = row[3]
    date =row[4]
    time = row[5]

    email_element = driver.find_element(By.CLASS_NAME, "email")
    password_element = driver.find_element(By.CLASS_NAME, "password")
    login = driver.find_element(By.CLASS_NAME, "loginx")    

    for ch in email:
        email_element.send_keys(ch)

    for ch in password:
        password_element.send_keys(ch)

    login.click()

    #redicting to admin-panel.php page
    WebDriverWait(driver, 10).until(EC.url_to_be("http://localhost:4000/admin-panel.php"))
    open_book_appointment = driver.find_element(By.CLASS_NAME, 'appointment')
    open_book_appointment.click()

    #choosing specilisation
    dropdown1=  WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'specialisation')))
    selected = Select(dropdown1)
    selected.select_by_visible_text(specialisation)

    #choosing doctor
    dropdown2 = driver.find_element(By.CLASS_NAME, 'doctor')
    selected = Select(dropdown2)
    selected.select_by_visible_text(doctor)

    #Entering date
    date_elem = driver.find_element(By.CLASS_NAME, 'date')
    date_elem.clear()
    date_Str = str(date)
    date_elem.send_keys(date_Str)

    #selecting time
    dropdown3 = driver.find_element(By.CLASS_NAME, 'time')
    selected3 = Select(dropdown3)
    time_str =str(time)
    selected3.select_by_value(time_str)

    #create new entry
    create = driver.find_element(By.CLASS_NAME, 'create')
    create.click()


    #Accepting the alert once after appointment sucessfull popup shows up
    WebDriverWait(driver, 10).until(EC.alert_is_present())
    alert = driver.switch_to.alert
    alert_text = alert.text
    alert.accept()
    
    #for counting succesfull and unsucessfull test cases!!
    expected_text = 'Your appointment successfully booked'
    if alert_text == expected_text:
        successfull+=1
    else:
        unsuccessfull+=1
        

    #lOG OUT
    logout = driver.find_element(By.CLASS_NAME, 'logout ')
    logout.click()
    


    #Back to login
    Back_login = driver.find_element(By.CLASS_NAME, 'BackToLogin')
    Back_login.click()

print(f'Total test case passed are {successfull}')
print(f'Total test cases failed are {unsuccessfull}')
conn.close()    
 


Total test case passed are 5
Total test cases failed are 0
